In [4]:
import cplex
from cplex.exceptions import CplexError
import sys
from __future__ import print_function

def smartStatement(graph, prob, ub):

    my_obj = []
    my_ub = []
    my_ctype = ""
    my_colnames = []
    my_rhs = []
    my_rownames = []
    my_sense = ""
    my_upperbound = ub
    print("greedy upperbound: ", my_upperbound)
    
    for color in range(0, my_upperbound):
        my_obj.append(1.0) # sum y_c
        my_ub.append(1.0) #all y_c are binary
        my_colnames.append("y"+str(color))
        my_ctype += "I" #all y_c are binary
    for vertex in range(0,len(graph)):
        my_rhs.append(1.0) #for sum_c x_{v,c} = 1 forall v
        my_sense += "E" 
        my_rownames.append("oneColor"+str(color))
    for vertex in range(0,len(graph)):
        for color in range(my_upperbound):
            my_obj.append(0.0) # x_{v,c}
            my_ub.append(1.0) #all x_{v,c} are binary
            my_colnames.append("x"+str(vertex)+","+str(color))
            my_ctype += "I" #all x_{v,c} are binary
            for adjVert in range(vertex+1,len(graph)):
                my_rhs.append(1.0) #for A[v,u](x_{v,c}+x_{u,c}) <= 1 forall v,u
                my_sense += "L" 
                my_rownames.append("noAdjCols"+str(vertex)+","+str(color)+","+str(adjVert))
    for vertex in range(len(graph)):
        for color in range(my_upperbound):
            my_rhs.append(0.0) #for x[v,c]-y[c] <= 0 forall v
            my_sense += "L"
            my_rownames.append("colorIsUsed"+str(color))
            
    
    prob.objective.set_sense(prob.objective.sense.minimize)
    # since lower bounds are all 0.0 (the default), lb is omitted here
    prob.variables.add(obj=my_obj, ub=my_ub, types=my_ctype,
                       names=my_colnames)
    rows =[]
    #sum_c x_{v,c} = 1 forall v
    for vertex in range(0,len(graph)):
        tmp1 = []
        tmp2 = []
        for color in range(0,my_upperbound):
            tmp1.append("x"+str(vertex)+","+str(color))
            tmp2.append(1.0)
        rows.append([tmp1,tmp2])
    #A[v,u](x_{v,c}+x_{u,c}) <= 1 forall v,u
    for vertex in range(0,len(graph)):
        for color in range(0,my_upperbound):
            for adjVert in range(vertex+1,len(graph)):
                #maybe Cplex hanles integers alright, but for now I made it long and ugly.
                rows.append([["x"+str(vertex)+","+str(color), "x"+str(adjVert)+","+str(color)],[graph[vertex][adjVert]*1.0,graph[adjVert][vertex]*1.0]])
    #x_{v,c}-y_c <= 0 forall v
    for vertex in range(0,len(graph)):
        for color in range(0,my_upperbound):
            rows.append([["y"+str(color),"x"+str(vertex)+","+str(color)],[-1.0,1.0]])
    #rownames might be removeable        
    prob.linear_constraints.add(lin_expr=rows, senses=my_sense,
                                rhs=my_rhs, names=my_rownames)
    
def statement(graph, prob):

    my_obj = []
    my_ub = []
    my_ctype = ""
    my_colnames = []
    my_rhs = []
    my_rownames = []
    my_sense = ""
    
    for color in range(0, len(graph)):
        my_obj.append(1.0)# sum y_c
        my_ub.append(1.0)#all y_c are binary
        my_colnames.append("y"+str(color))
        my_ctype += "I" #all y_c are binary
        my_rhs.append(1.0) #sum_c x_{v,c} = 1 forall v
        my_sense += "E" 
        my_rownames.append("oneColor"+str(color))
    for vertex in range(0,len(graph)):
        for color in range(0,len(graph)):
            my_obj.append(0.0)
            my_ub.append(1.0)
            my_colnames.append("x"+str(vertex)+","+str(color))
            my_ctype += "I" #all x_{v,c} are binary
            for adjVert in range(vertex+1,len(graph)):
                my_rhs.append(1.0) #for A[v,u](x_{v,c}+x_{u,c}) <= 1 forall v,u
                my_sense += "L" 
                my_rownames.append("noAdjCols"+str(vertex)+","+str(color)+","+str(adjVert))
    for vertex in range(0,len(graph)):
        for color in range(0,len(graph)):
            my_rhs.append(0.0) #for x_{v,c}-y_c <= 0 forall v
            my_sense += "L"
            my_rownames.append("colorIsUsed"+str(color))
            
    
    prob.objective.set_sense(prob.objective.sense.minimize)
    # since lower bounds are all 0.0 (the default), lb is omitted here
    prob.variables.add(obj=my_obj, ub=my_ub, types=my_ctype,
                       names=my_colnames)
    rows =[]
    #sum_c(x_{v,c}) = 1 forall v
    for vertex in range(0,len(graph)):
        tmp1 = []
        tmp2 = []
        for color in range(0,len(graph)):
            tmp1.append("x"+str(vertex)+","+str(color))
            tmp2.append(1.0)
        rows.append([tmp1,tmp2])
    #A[v,u](x_{v,c}+x_{u,c}) <= 1 forall v,u
    for vertex in range(0,len(graph)):
        for color in range(0,len(graph)):
            for adjVert in range(vertex+1,len(graph)):
                #maybe Cplex hanles integers alright, but for now i made it long and ugly.
                rows.append([["x"+str(vertex)+","+str(color), "x"+str(adjVert)+","+str(color)],[graph[vertex][adjVert]*1.0,graph[adjVert][vertex]*1.0]])
    #x_{v,c}-y_c <= 0 forall v
    for vertex in range(0,len(graph)):
        for color in range(0,len(graph)):
            rows.append([["y"+str(color),"x"+str(vertex)+","+str(color)],[-1.0,1.0]])
    #rownames might be removeable        
    prob.linear_constraints.add(lin_expr=rows, senses=my_sense,
                                rhs=my_rhs, names=my_rownames)
    
    
def SchedulingForm(graph, prob, ub):
    my_obj = []
    my_ub = []
    my_ctype = ""
    my_colnames = []
    my_rhs = []
    my_rownames = []
    my_sense = ""
    my_upperbound = ub
    print("greedy upperbound: ", my_upperbound)
    
    for vertex in range(len(graph)):
        my_obj.append(0.0)
        my_ub.append(ub)
        my_colnames.append("X"+str(vertex))
        my_ctype += "I" #all colors are integral
        
        my_rhs.append(-1.0)
        my_sense += "L"
        my_rownames.append("less than highest color "+str(vertex))
    for vertex in range(len(graph)):
        for edge in range(vertex):
            if graph[vertex][edge] == 1:
                my_obj.append(0.0)
                my_ub.append(1.0) #x_{u,v} is binary
                my_colnames.append("x"+str(vertex)+","+str(edge))
                my_ctype += "I" #x_{u,v} is binary
                
                my_rhs.append(my_upperbound-1.0) #X_u - X_v + Kx_{u,v} =< K-1  , K >= c
                my_sense += "L" 
                my_rownames.append("1 nonAdj"+str(vertex)+","+str(edge))
                my_rhs.append(-1.0) #X_v - X_u - Kx_{u,v} =< -1
                my_sense += "L" 
                my_rownames.append("2 nonAdj"+str(vertex)+","+str(edge))
                
    my_obj.append(1.0)
    my_ub.append(ub)
    my_colnames.append("c")
    my_ctype += "I" #c is integral (but it will be even when real)
            
    
    prob.objective.set_sense(prob.objective.sense.minimize)
    # since lower bounds are all 0.0 (the default), lb is omitted here
    prob.variables.add(obj=my_obj, ub=my_ub, types=my_ctype,
                       names=my_colnames)
    rows =[]
    #X_v - c =< 0 forall v
    for vertex in range(len(graph)):
        rows.append([["c","X"+str(vertex)],[-1.0,1.0]])

    for vertex in range(len(graph)):
        for edge in range(vertex):
            if graph[vertex][edge] == 1:
                #X_u - X_v + Kx_{u,v} =< K-1  , K >= c
                rows.append([["X"+str(vertex),"X"+str(edge),"x"+str(vertex)+","+str(edge)],[1.0,-1.0,my_upperbound]])
                #X_v - X_u - Kx_{u,v} =< -1
                rows.append([["X"+str(edge),"X"+str(vertex),"x"+str(vertex)+","+str(edge)],[1.0,-1.0,-my_upperbound]])    
    prob.linear_constraints.add(lin_expr=rows, senses=my_sense,
                                rhs=my_rhs, names=my_rownames)
    
    
def cromNum(graph, method):
    for row in range (1,len(graph)):
        if len(graph[row-1]) != len(graph[row]):
            print("Input a proper adjacency matrix")
            return
    try:
        my_prob = cplex.Cplex()
        my_prob.parameters.timelimit.set(60.0)
        my_prob.parameters.timelimit.get()
        if method == "smart":
            ub = greedyColor(graph)
            handle = smartStatement(graph, my_prob, ub)
        elif method == "Scheduling":
            ub = greedyColor(graph)
            handle = SchedulingForm(graph, my_prob, ub)
        else:
            ub = len(graph)
            handle = statement(graph, my_prob)
        my_prob.solve()
    except CplexError as exc:
            print(exc)
            return

    print()
    # solution.get_status() returns an integer code
    print("Solution status = ", my_prob.solution.get_status(), ":", end= '')
    # the following line prints the corresponding string
    print(my_prob.solution.status[my_prob.solution.get_status()])
    print("Solution value  = ", my_prob.solution.get_objective_value())
    print("lower bound  = ", my_prob.solution.MIP.get_best_objective())

    
    # Print result:
    if method == "Scheduling":
        vertices = my_prob.solution.get_values(0, len(graph))
        cromnum = my_prob.solution.get_objective_value()
        for color in range(int(round(cromnum))):
            colGroup = []
            for i in range(len(graph)):
                if int(round(vertices[i])) == color:
                    colGroup.append(i + 1)
            print("vertices with colour "+str(color+1)+":", colGroup)
    else:
        colors = my_prob.solution.get_values(0, ub)
        nextColor = 1
        for color in range(ub):
            colGroup = []
            if colors[color] > 0.5:
                for vertex in range(len(graph)):
                    if my_prob.solution.get_values("x"+str(vertex)+","+str(color)) > 0.5:
                        colGroup.append(vertex+1)
            if colGroup != []:
                print("vertices with colour "+str(nextColor)+":", colGroup)
                nextColor += 1
            
            

#    print(x)
#    for j in range(numrows):
#        print("Row %d:  Slack = %10f" % (j, slack[j]))
#    for j in range(numcols):
#        print("Column %d:  Value = %10f" % (j, x[j]))

In [5]:
def greedyColor(graph):
    vertices = len(graph)
    colors = [0]*vertices
    colors[0] = 1
    for vertex in range(vertices):
        neighbours = []
        for neighbour in range(vertex+1):
            if graph[vertex][neighbour] == 1:
                neighbours.append(neighbour)
        for c in range(vertices+1):
            if not any([colors[n] == c for n in neighbours]):
                break
        colors[vertex] = c
    return max(colors)+1

In [8]:
# Downloaded Graphs:
# queen5_5.col
# queen6_6.col
# queen7_7.col
# queen8_8.col
# queen8_12.col
# queen9_9.col
# myciel3.col
# homer.col
with open("myciel3.col", "r") as ins:
    for lines in ins:
        if lines[0] == "p":
            nodes = int(lines.split()[2])
            adjArr = [[0 for i in range(nodes)] for j in range(nodes)]
        if lines[0] == "e":
            v1 = int(lines.split()[1])-1
            v2 = int(lines.split()[2])-1
            adjArr[v1][v2] = adjArr[v2][v1] = 1

#cromNum(adjArr, "")
#cromNum(adjArr, "smart") 
cromNum(adjArr, "Scheduling")

Tried aggregator 1 time.
MIP Presolve eliminated 506 rows and 0 columns.
MIP Presolve modified 220 coefficients.
Reduced MIP has 231 rows, 132 columns, and 781 nonzeros.
Reduced MIP has 132 binaries, 0 generals, 0 SOSs, and 0 indicators.
Presolve time = 0.01 sec. (0.68 ticks)
Found incumbent of value 11.000000 after 0.01 sec. (1.32 ticks)
Probing time = 0.00 sec. (0.14 ticks)
Tried aggregator 1 time.
Reduced MIP has 231 rows, 132 columns, and 781 nonzeros.
Reduced MIP has 132 binaries, 0 generals, 0 SOSs, and 0 indicators.
Presolve time = 0.01 sec. (0.49 ticks)
Probing time = 0.00 sec. (0.14 ticks)
Clique table members: 231.
MIP emphasis: balance optimality and feasibility.
MIP search method: dynamic search.
Parallel mode: deterministic, using up to 4 threads.
Root relaxation solution time = 0.00 sec. (1.59 ticks)

        Nodes                                         Cuts/
   Node  Left     Objective  IInf  Best Integer    Best Bound    ItCnt     Gap

*     0+    0                    

In [7]:
SoerenGraph =[[0,0,0,0,1,1,0,1,1,1,1,1,0,0,1,0,0,1,0,1,0,0,0,0,1,0,1,0,0,1,1,1],
              [0,0,1,1,1,0,0,1,0,0,1,1,1,0,0,1,1,1,0,0,0,0,1,0,1,0,0,1,0,0,1,1],
              [0,1,0,0,0,0,1,1,1,0,1,1,1,1,0,0,0,0,0,1,0,1,0,0,1,1,1,0,1,0,0,1],
              [0,1,0,0,1,1,1,0,0,1,0,0,1,1,1,0,1,0,1,1,0,0,0,0,1,1,1,0,0,1,0,0],
              [1,1,0,1,0,0,0,0,0,0,1,0,1,1,1,1,0,0,0,0,0,1,0,1,0,0,1,1,1,0,1,0],
              [1,0,0,1,0,0,1,1,1,0,0,1,0,0,1,1,0,0,1,0,1,1,0,0,0,0,1,1,1,0,0,1],
              [0,0,1,1,0,1,0,0,1,1,0,0,1,0,1,1,0,0,0,0,0,0,0,1,0,0,0,0,1,1,1,0],
              [1,1,1,0,0,1,0,0,1,1,1,0,0,1,0,0,0,0,0,0,1,0,1,1,0,0,0,0,1,1,1,0],
              [1,0,1,0,0,1,1,1,0,0,0,0,1,1,0,1,1,1,1,1,0,0,1,0,0,1,0,1,0,0,0,0],
              [1,0,0,1,0,0,1,1,0,0,1,1,1,0,0,1,0,0,1,1,1,0,0,1,1,1,0,0,0,0,1,0],
              [1,1,1,0,1,0,0,1,0,1,0,0,0,0,1,1,1,0,1,1,1,1,0,0,0,0,0,1,0,1,0,0],
              [1,1,1,0,0,1,0,0,0,1,0,0,1,1,1,0,0,1,0,0,1,1,1,0,1,0,1,1,0,0,0,0],
              [0,1,1,1,1,0,1,0,1,1,0,1,0,0,0,0,0,0,1,0,1,1,1,1,0,0,0,0,0,1,0,1],
              [0,0,1,1,1,0,0,1,1,0,0,1,0,0,1,1,1,0,0,1,0,0,1,1,0,0,1,0,1,1,0,0],
              [1,0,0,1,1,1,1,0,0,0,1,1,0,1,0,0,1,1,0,0,1,0,1,1,0,0,0,0,0,0,0,1],
              [0,1,0,0,1,1,1,0,1,1,1,0,0,1,0,0,1,1,1,0,0,1,0,0,0,0,0,0,1,0,1,1],
              [0,1,0,1,0,0,0,0,1,0,1,0,0,1,1,1,0,0,0,0,1,1,0,1,1,1,1,1,0,0,1,0],
              [1,1,0,0,0,0,0,0,1,0,0,1,0,0,1,1,0,0,1,1,1,0,0,1,0,0,1,1,1,0,0,1],
              [0,0,0,1,0,1,0,0,1,1,1,0,1,0,0,1,0,1,0,0,0,0,1,1,1,0,1,1,1,1,0,0],
              [1,0,1,1,0,0,0,0,1,1,1,0,0,1,0,0,0,1,0,0,1,1,1,0,0,1,0,0,1,1,1,0],
              [0,0,0,0,0,1,0,1,0,1,1,1,1,0,1,0,1,1,0,1,0,0,0,0,0,0,1,0,1,1,1,1],
              [0,0,1,0,1,1,0,0,0,0,1,1,1,0,0,1,1,0,0,1,0,0,1,1,1,0,0,1,0,0,1,1],
              [0,1,0,0,0,0,0,1,1,0,0,1,1,1,1,0,0,0,1,1,0,1,0,0,1,1,0,0,1,0,1,1],
              [0,0,0,0,1,0,1,1,0,1,0,0,1,1,1,0,1,1,1,0,0,1,0,0,1,1,1,0,0,1,0,0],
              [1,1,1,1,0,0,0,0,0,1,0,1,0,0,0,0,1,0,1,0,0,1,1,1,0,0,0,0,1,1,0,1],
              [0,0,1,1,0,0,0,0,1,1,0,0,0,0,0,0,1,0,0,1,0,0,1,1,0,0,1,1,1,0,0,1],
              [1,0,1,1,1,1,0,0,0,0,0,1,0,1,0,0,1,1,1,0,1,0,0,1,0,1,0,0,0,0,1,1],
              [0,1,0,0,1,1,0,0,1,0,1,1,0,0,0,0,1,1,1,0,0,1,0,0,0,1,0,0,1,1,1,0],
              [0,0,1,0,1,1,1,1,0,0,0,0,0,1,0,1,0,1,1,1,1,0,1,0,1,1,0,1,0,0,0,0],
              [1,0,0,1,0,0,1,1,0,0,1,0,1,1,0,0,0,0,1,1,1,0,0,1,1,0,0,1,0,0,1,1],
              [1,1,0,0,1,0,1,1,0,1,0,0,0,0,0,1,1,0,0,1,1,1,1,0,0,0,1,1,0,1,0,0],
              [1,1,1,0,0,1,0,0,0,0,0,0,1,0,1,1,0,1,0,0,1,1,1,0,1,1,1,0,0,1,0,0]]
#cromNum(SoerenGraph, "smart")
cromNum(SoerenGraph, "Scheduling")

greedy upperbound:  6
Tried aggregator 1 time.
Reduced MIP has 500 rows, 267 columns, and 1468 nonzeros.
Reduced MIP has 234 binaries, 33 generals, 0 SOSs, and 0 indicators.
Presolve time = 0.01 sec. (0.76 ticks)
Found incumbent of value 6.000000 after 0.01 sec. (2.17 ticks)
Probing fixed 0 vars, tightened 1 bounds.
Probing time = 0.00 sec. (0.68 ticks)
Tried aggregator 1 time.
Reduced MIP has 500 rows, 267 columns, and 1468 nonzeros.
Reduced MIP has 234 binaries, 33 generals, 0 SOSs, and 0 indicators.
Presolve time = 0.00 sec. (0.68 ticks)
Probing time = 0.00 sec. (0.68 ticks)
MIP emphasis: balance optimality and feasibility.
MIP search method: dynamic search.
Parallel mode: deterministic, using up to 4 threads.
Root relaxation solution time = 0.00 sec. (1.02 ticks)

        Nodes                                         Cuts/
   Node  Left     Objective  IInf  Best Integer    Best Bound    ItCnt     Gap

*     0+    0                            6.0000        2.0000            66.67%
 